# Generating DEA Intertidal Elevation

This notebook demonstrates how to generate DEA Intertidal Elevation outputs on
a combination of Landsat and Sentinel-2 data.

**EDIT: modified from the original, here: https://github.com/GeoscienceAustralia/dea-intertidal/blob/rbt/notebooks/Intertidal_elevation_stac.ipynb**

## Getting started

Install additional packages, which may not be in the environment you have ready.

Make sure the kernel you're using is the custom one we made at the start of the workshop.

In [ ]:
%pip install --upgrade git+https://github.com/GeoscienceAustralia/dea-intertidal.git pyTMD dea-tools

### Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from ipyleaflet import basemaps, basemap_to_tiles

import odc.geo.xr  # noqa: F401
from odc.ui import select_on_a_map
from dea_tools.dask import create_local_dask_cluster

from intertidal.io import prepare_for_export
from intertidal.elevation import elevation

from odc.stac import load, configure_s3_access
from pystac_client import Client
from odc.geo.geom import BoundingBox

### Configuration

Set up some study parameters. You don't need to change this, but if you want to run over a longer time
period, or iwth a different tide model, you can set that up here.

In [ ]:
# Configure study parameters
start_date = "2020"  # Start date for analysis
end_date = "2022"  # End date for analysis
resolution = 30  # Spatial resolution used for output files
ndwi_thresh = 0.1  # Threshold used to identify dry/wet transition

# Tide data
tide_model = "FES2022"  # Tide model to use in analysis
home = os.path.expanduser("~")  # Home directory
tide_model_dir = home + "/tide_models"  # Directory containing tide model files

### Study site selection

Pick a region to run the model over. There's a section of the Vietnam coast that works well to the east of Nam Dinh.

In [ ]:
# Set study area name for outputs
study_area = "se-asia"

# Plot interactive map to select area
basemap = basemap_to_tiles(basemaps.Esri.WorldImagery)
geom = select_on_a_map(height="600px", layers=(basemap,), center=(12, 120), zoom=4)
geom

## Find and load data

First we use the Element-84 STAC API to search for Landsat data from the USGS and
then we use the same API to search for Sentinel-2 data from the AWS open data program.

Once we have STAC Items, we load this data and mask clouds out, before calculating
the NDWI index, which is used to compute a wet/dry threshold.

In [ ]:
# We're using the Element-84 STAC API, as it provides access to both Landsat and Sentinel-2 records
catalog = "https://earth-search.aws.element84.com/v1"
datetime = f"{start_date}/{end_date}"
landsat_collection = "landsat-c2-l2"
sentinel2_collection = "sentinel-2-l2a"

# Connect to client
client = Client.open(catalog)

# Search for Landsat items
landsat_items = client.search(
    collections=[landsat_collection],
    intersects=geom,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 50}},
).item_collection()

# Search for Sentinel-2 items
sentinel2_items = client.search(
    collections=[sentinel2_collection],
    intersects=geom,
    datetime=datetime,
    query={"eo:cloud_cover": {"lt": 50}},
).item_collection()

# Check how many items were returned
print(f"Found {len(landsat_items)} Landsat items and {len(sentinel2_items)} Sentinel-2 items")

In [ ]:
common_options = {
    "chunks": {"x": 2048, "y": 2048},
    "groupby": "solar_day",
    "resampling": {"qa_pixel": "nearest", "SCL": "nearest", "*": "cubic"},
    "resolution": 30
}

# Load Landsat with ODC STAC
landsat_data = load(
    items=landsat_items,
    geopolygon=geom,
    bands=["green", "nir08", "qa_pixel"],
    **common_options,
)

# Apply Landsat cloud mask
# Bit flag mask for the QA_PIXEL band
# We need Bit 3: high confidence cloud, bit 4: high confidence shadow,
# which are the 4th and 5th bits from the right (0-indexed)
bitflags = 0b00011000

cloud_mask = (landsat_data.qa_pixel & bitflags) != 0
masked = landsat_data.where(~cloud_mask).drop_vars("qa_pixel")

sentinel2_data = load(
    items=sentinel2_items,
    geopolygon=geom,
    bands=["green", "nir", "scl"],
    **common_options,
)

# Apply Sentinel-2 cloud mask
# 1: defective, 3: shadow, 9: high confidence cloud
mask_flags = [1, 3, 9]

cloud_mask = ~sentinel2_data.scl.isin(mask_flags)
sentinel2_data = sentinel2_data.where(cloud_mask).drop_vars("scl")

# Apply scaling
ds_ls = (landsat_data.where(landsat_data != 0) * 0.0000275 + -0.2).clip(0, 1)
ds_s2 = (sentinel2_data.where(sentinel2_data != 0) * 0.0001).clip(0, 1)

# Convert to NDWI
ndwi_ls = (ds_ls.green - ds_ls.nir08) / (ds_ls.green + ds_ls.nir08)
ndwi_s2 = (ds_s2.green - ds_s2.nir) / (ds_s2.green + ds_s2.nir)

# Combine into a single dataset
all_data = (
    xr.concat([ndwi_ls, ndwi_s2], dim="time").sortby("time").to_dataset(name="ndwi")
)

all_data

## Run the analysis

Up to this point, we've been "lazy loading" our data, using Dask to structure
our data processing plan. The below cells actually load data into memory. and
undertake processing.

First we need to set up a local Dask cluster, and configure S3 access, and then
we can run the model

In [ ]:
# Create local dask cluster to improve data load time. Only run this once.
dask_client = create_local_dask_cluster(return_client=True)

# Configure S3 access. Cloud defaults is an optimisation, while requester pays is required for Landsat
configure_s3_access(cloud_defaults=True, requester_pays=True)

### Generate Intertidal elevation

In [ ]:
# Model elevation for each pixel
ds, _ = elevation(
    all_data,
    tide_model=tide_model,
    tide_model_dir=tide_model_dir,
)

### Plot outputs

In [ ]:
fix, axes = plt.subplots(1, 4, figsize=(12, 3))
ds.elevation.plot.imshow(cmap="viridis", ax=axes[0])
ds.elevation_uncertainty.plot.imshow(cmap="inferno", ax=axes[1])
ds.qa_ndwi_corr.plot.imshow(cmap="RdBu", vmin=-0.7, vmax=0.7, ax=axes[2])
ds.qa_ndwi_freq.plot.imshow(cmap="Blues", vmin=0, vmax=1, ax=axes[3])

In [ ]:
ds.elevation.odc.explore(
    robust=True,
    tiles=basemaps.Esri.WorldImagery
)

### Export to GeoTIFF files

In [ ]:
# Create output folder if it doesn't exist
output_dir = f"data/interim/{study_area}/{start_date}-{end_date}"
os.makedirs(output_dir, exist_ok=True)

# Prepare data for export
ds["qa_ndwi_freq"] *= 100  # Convert frequency to %

# Prepare for export by correctly setting nodata and dtypes
ds_prepared = prepare_for_export(ds, output_location=output_dir)

### Close Dask client

In [ ]:
dask_client.close()